<a href="https://colab.research.google.com/github/mrabhishekmore/colab_data/blob/main/pyspark_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyspark==3.4.1 spark-nlp==5.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.3/536.3 kB 27.4 MB/s eta 0:00:00


In [18]:
from pyspark.sql import SparkSession

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Get values as list") \
    .getOrCreate()

# Sample data
data = [("John", "Math"),
        ("Alice", "Physics"),
        ("Bob", "Chemistry")]

# Create a DataFrame
df = spark.createDataFrame(data, ["name", "sub"])

# Collect the values of the "sub" column into a Python list
sub_list = df.select("sub").rdd.flatMap(lambda x: x).collect()


# Show the list
print(sub_list)


['Math', 'Physics', 'Chemistry']


In [17]:
from pyspark.sql.functions import array, explode, expr, col, when, concat, coalesce
# df = df.withColumn("joined",concat(col("name"),col("sub")))

# df.show()

df.select(coalesce(df["sub"],df["name"])).show()

+-------------------+
|coalesce(sub, name)|
+-------------------+
|               Math|
|              Alice|
|                Bob|
+-------------------+



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array, explode, expr, col, when

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Explode Columns") \
    .getOrCreate()

# Sample data
data = [("John", 70, 80, 90),
        ("Alice", 75, 85, 95),
        ("Bob", 80, 90, 85)]

# Create a DataFrame
df = spark.createDataFrame(data, ["name", "sub1", "sub2", "sub3"])

print("Source df")

df.show()

sub_cols = ["sub1", "sub2", "sub3"]
# Collect sub1, sub2, and sub3 into an array column
df = df.withColumn("subjects", array(*sub_cols))

# Explode the array column into separate rows
result_df = df.select("name", explode("subjects").alias("score"))

result_df = result_df.withColumn("grade", expr("CASE WHEN score > 80 THEN 'PASS' ELSE 'FAIL' END"))

result_df = result_df.withColumn("result",when(result_df["score"] > 80, "pass").otherwise("fail"))

# result_df = result_df.filter(col("grade")=="PASS")
# Show the result
result_df.show()


Source df
+-----+----+----+----+
| name|sub1|sub2|sub3|
+-----+----+----+----+
| John|  70|  80|  90|
|Alice|  75|  85|  95|
|  Bob|  80|  90|  85|
+-----+----+----+----+

+-----+-----+-----+------+
| name|score|grade|result|
+-----+-----+-----+------+
| John|   70| FAIL|  fail|
| John|   80| FAIL|  fail|
| John|   90| PASS|  pass|
|Alice|   75| FAIL|  fail|
|Alice|   85| PASS|  pass|
|Alice|   95| PASS|  pass|
|  Bob|   80| FAIL|  fail|
|  Bob|   90| PASS|  pass|
|  Bob|   85| PASS|  pass|
+-----+-----+-----+------+



In [4]:
sorted_df = result_df.sort(result_df.score.desc())
sorted_df.show()

+-----+-----+-----+
| name|score|grade|
+-----+-----+-----+
|Alice|   95| PASS|
| John|   90| PASS|
|  Bob|   90| PASS|
|Alice|   85| PASS|
|  Bob|   85| PASS|
| John|   80| FAIL|
|  Bob|   80| FAIL|
|Alice|   75| FAIL|
| John|   70| FAIL|
+-----+-----+-----+



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat_ws, collect_list, count

# Create a SparkSession
spark = SparkSession.builder \
    .appName("Concatenate IDs by Name") \
    .getOrCreate()

# Sample data
data = [("John", 1),
        ("Alice", 2),
        ("John", 3),
        ("Bob", 4),
        ("Alice", 5)]

# Create a DataFrame
df = spark.createDataFrame(data, ["name", "id"])

# Group by "name" and concatenate "id" with "|" delimiter
result_df = df.groupBy("name") \
              .agg(concat_ws("|", collect_list("id")).alias("concatenated_ids"))

count_df = df.groupBy("name") \
              .agg(count("*").alias("count")

# Show the result
result_df.show()

count_df.show()


+-----+----------------+
| name|concatenated_ids|
+-----+----------------+
| John|             1|3|
|Alice|             2|5|
|  Bob|               4|
+-----+----------------+

+-----+-----+
| name|count|
+-----+-----+
| John|    2|
|Alice|    2|
|  Bob|    1|
+-----+-----+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Create a Spark session
spark = SparkSession.builder.appName("example").getOrCreate()

# Dummy data
data = [("card|spec|65",),
        ("fam|tax|87",)]

# Create DataFrame
df = spark.createDataFrame(data, ["specialty"])

# Show the DataFrame
df.show()

+------------+
|   specialty|
+------------+
|card|spec|65|
|  fam|tax|87|
+------------+

